In [1]:
# Looking at population pyramids and IFR
# Method (and some of the code) based on Marc Bevand's work, see https://github.com/mbevand/covid19-age-stratified-ifr 

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)

import requests

import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
fs_label = 16
parameters = {
                'axes.labelsize': fs_label,
                'axes.titlesize': fs_label+4,
                'xtick.labelsize': fs_label,
                'ytick.labelsize': fs_label, 
                'legend.fontsize': fs_label, 
                'lines.markersize': 10,
                'lines.linewidth': 3
             }
plt.rcParams.update(parameters)
%matplotlib widget
import matplotlib.colors as colors
from matplotlib import cm # Colormaps

cmap = cm.get_cmap('Dark2',7)

import locale
import matplotlib.dates as mdates
# locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))


import os
import math

from datetime import date
import datetime

saveFigures = True
print('saveFigures is set to: '+str(saveFigures))
print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]
def rnTime2(t,meanWidth):
    return t[meanWidth-1:]
def plotWithMean(ax,x,y,meanWidth):
    firstLine = ax.plot(x,y,'.:',markersize=2,linewidth=0.5)
    firstColor = firstLine[0].get_color()
    ax.plot(rnTime(x,meanWidth),rnMean(y,meanWidth),color=firstColor)

saveFigures is set to: True
Done loading packages


In [2]:
# Pyramid data is from the United Nations: this file is a CSV export of the first sheet
# of "Population by Age Groups - Both Sexes" linked from:
# https://population.un.org/wpp/Download/Standard/Population/
# Direct link:
# https://population.un.org/wpp/Download/Files/1_Indicators%20(Standard)/EXCEL_FILES/1_Population/WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.xlsx
file_pyramids = 'data/WPP2019_POP_F07_1_POPULATION_BY_AGE_BOTH_SEXES.csv'

maxage = 100
# Age groups defined in the CSV file
age_groups = [(0,4), (5,9), (10,14), (15,19), (20,24), (25,29), (30,34), (35,39), (40,44), (45,49), (50,54), (55,59), (60,64), (65,69), (70,74), (75,79), (80,84), (85,89), (90,94), (95,99), (100,maxage)]

# This will hold parsed pyramid data. Example to get the number of people in the
# age group 20-24 in France: pyramid['France'][(20,24)]
pyramid = {}

def ag2str(age_group):
    if age_group[1] == maxage:
        return f'{age_group[0]}+'
    return f'{age_group[0]}-{age_group[1]}'

def parse_pyramids():
    df = pd.read_csv(file_pyramids)
    # ignore labels as they don't contain any data
    df = df[df['Type'] != 'Label/Separator']
    # only take rows with data as of 2020
    df = df[df['Reference date (as of 1 July)'] == 2020]
    # only parse countries, world, and continents
    df = df[df['Type'].isin(('Country/Area', 'World', 'Region'))]
    # remove spaces used as thousands separators, and convert cell values to floats
    columns = [ag2str(x) for x in age_groups]
    for col in columns:
        df[col] = df[col].str.replace('\s+', '').astype(float)
    regions = list(df['Region, subregion, country or area *'])
    #regions = ('France',)
    for region in regions:
        pyramid[region] = {}
        df_region = df[df['Region, subregion, country or area *'] == region]
        for ag in age_groups:
            # values are in thousands
            pyramid[region][ag] = 1000 * float(df_region[ag2str(ag)])

In [3]:

# For a description of cdc_sympt, see the same variable name defined in covid_vs_flu.py
cdc_sympt = .33

# Various age-stratified IFR estimates
ifrs = [

        # Calculated from Spanish ENE-COVID study
        # (see calc_ifr.py)
        ('ENE-COVID', {
            (0,9):    0.003,
            (10,19):  0.004,
            (20,29):  0.015,
            (30,39):  0.030,
            (40,49):  0.064,
            (50,59):  0.213,
            (60,69):  0.718,
            (70,79):  2.384,
            (80,89):  8.466,
            (90,maxage): 12.497,
        }),

        # US CDC estimate as of 19 Mar 2021
        # https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html
        # (table 1)
        ('COVID: US CDC', {
            (0,17):   0.002,
            (18,49):  0.05,
            (50,64):  0.6,
            (65,maxage): 9.0,
        }),

        # Verity et al.
        # https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30243-7/fulltext
        # (table 1)
        ('COVID: Verity', {
            (0,9):    0.00161,
            (10,19):  0.00695,
            (20,29):  0.0309,
            (30,39):  0.0844,
            (40,49):  0.161,
            (50,59):  0.595,
            (60,69):  1.93,
            (70,79):  4.28,
            (80,maxage): 7.80,
        }),

        # Levin et al.
        # https://link.springer.com/article/10.1007/s10654-020-00698-1
        # (table 3)
        ('COVID: Levin', {
            (0,34):   0.004,
            (35,44):  0.068,
            (45,54):  0.23,
            (55,64):  0.75,
            (65,74):  2.5,
            (75,84):  8.5,
            (85,maxage): 28.3,
        }),

        # Brazeau et al.
        # https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/report-34-ifr/
        # (table 2, column "IFR (%) with Seroreversion")
        ('COVID: Brazeau', {
            (0,4):    0.00,
            (5,9):    0.01,
            (10,14):  0.01,
            (15,19):  0.02,
            (20,24):  0.02,
            (25,29):  0.04,
            (30,34):  0.06,
            (35,39):  0.09,
            (40,44):  0.15,
            (45,49):  0.23,
            (50,54):  0.36,
            (55,59):  0.57,
            (60,64):  0.89,
            (65,69):  1.39,
            (70,74):  2.17,
            (75,79):  3.39,
            (80,84):  5.30,
            (85,89):  8.28,
            (90,maxage): 16.19,
        }),

        # IFR for seasonal influenza
        # US CDC 2019-2020 influenza burden
        # https://www.cdc.gov/flu/about/burden/2019-2020.html
        ('Flu: US CDC', {
            (0,4):              254/4_291_677 * 100 * cdc_sympt,
            (5,17):             180/8_214_257 * 100 * cdc_sympt,
            (18,49):            2_669/15_325_708 * 100 * cdc_sympt,
            (50,64):            5_133/8_416_702 * 100 * cdc_sympt,
            (65,maxage):        13_673/1_946_161 * 100 * cdc_sympt,
        }),

]


In [4]:
parse_pyramids()
# pyramid

<ipython-input-2-80e8f36a4a51>:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace('\s+', '').astype(float)


In [5]:

def people_of_age(pyramid_region, age):
    # Returns the number of people of exact age 'age', given the provided age pyramid
    for ((a, b), n) in pyramid_region.items():
        if age in range(a, b + 1):
            return n / float(b - a + 1)


In [6]:
people_of_age(pyramid['Denmark'],80) 

29400.0

In [7]:

def overall_ifr(pyramid_region, ifr_age_stratified):
    pop = 0
    deaths = 0
    for (age_group, ifr) in ifr_age_stratified.items():
        for age in range(age_group[0], age_group[1] + 1):
            pop += people_of_age(pyramid_region, age)
            deaths += people_of_age(pyramid_region, age) * ifr / 100.0
            
    # print(pop)
    # print(sum(pyramid_region.values()))
    # assert pop == sum(pyramid_region.values())
    return 100.0 * deaths / pop

In [8]:
overall_ifr(pyramid_region=pyramid['Denmark'],ifr_age_stratified=ifrs[0][1])

0.7970310827145569

# For check of method

In [9]:
allDKifrs = []
for k in range(0,5):
    allDKifrs.append(overall_ifr(pyramid_region=pyramid['Denmark'],ifr_age_stratified=ifrs[k][1]))
    
allDKifrs

# From Marc Bezand's table: | 0.797 | 1.954 | 1.136 | 1.593 | 0.878 |

[0.7970310827145569,
 1.9538892764634779,
 1.1364128924192711,
 1.592907269901572,
 0.8775841823519256]

In [10]:
# pyramid

# Vaccine-dependent IFR

In [11]:
# pyramid['Denmark']

dkDefPyr = pyramid['Denmark']
# dkDefPyr = pyramid['Peru']
# dkDefPyr = pyramid['Latin America and the Caribbean']
print(dkDefPyr)
fullPop = np.fromiter(dkDefPyr.values(), dtype=float).sum()

# keys = np.array(list(dkDefPyr.keys()))
# values = np.array(list(dkDefPyr.values()))
keys = list(dkDefPyr.keys())
# newPyr = {keys:values}


# ratioVacc = 0.99

allVaccRate = np.arange(0,0.99,0.01)
# allVaccRate = np.arange(0,0.99,0.2)
allIFRs = []

for ratioVacc in allVaccRate:
    curPyr = dkDefPyr.copy()
    toRemoveTot = np.ceil(fullPop * (ratioVacc))
    toRemove = toRemoveTot
    for k in range(len(curPyr)-1,-1,-1):
        thisKey = keys[k]
        thisCount = curPyr[thisKey]
        
        if (toRemove >= thisCount):
            curPyr[thisKey] = 0
            toRemove -= thisCount
        else:
            curPyr[thisKey] -= toRemove
            # curPyr[thisKey] = np.floor(curPyr[thisKey] - toRemove)
            toRemove = 0
        
        
        # print(curPyr[keys[k]])
    # print(curPyr)
    # print(overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[1][1]))
    curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[0][1])
    
    allIFRs.append(curIFR)

{(0, 4): 309000.0, (5, 9): 297000.0, (10, 14): 337000.0, (15, 19): 339000.0, (20, 24): 374000.0, (25, 29): 402000.0, (30, 34): 355000.0, (35, 39): 317000.0, (40, 44): 361000.0, (45, 49): 377000.0, (50, 54): 422000.0, (55, 59): 388000.0, (60, 64): 346000.0, (65, 69): 309000.0, (70, 74): 351000.0, (75, 79): 235000.0, (80, 84): 147000.0, (85, 89): 78000.0, (90, 94): 36000.0, (95, 99): 10000.0, (100, 100): 1000.0}


In [12]:
fig,ax1 = plt.subplots()
allAllIFRs = []

lstyles = ('solid', 'dashed', 'dotted', 'dashdot')
lstyles = ('dashed', 'dotted', 'dashdot')
markers = ('o', 's', 'v', '^', '<', '>', 'P', '*', 'X', 'D', 'p')
# cmap = plt.cm.twilight()
def getColor(i):
    # return plt.cm.twilight((1/5) * i)
    return plt.cm.bwr((1/10) * i)

counter = 0
for ifrDatabase in range(0,5):

    allVaccRate = np.arange(0,1.0,0.01)
    # allVaccRate = np.arange(0,0.99,0.2)
    allIFRs = []

    for ratioVacc in allVaccRate:
        curPyr = dkDefPyr.copy()
        toRemoveTot = np.ceil(fullPop * (ratioVacc))
        toRemove = toRemoveTot
        for k in range(len(curPyr)-1,-1,-1):
            thisKey = keys[k]
            thisCount = curPyr[thisKey]
            
            if (toRemove >= thisCount):
                curPyr[thisKey] = 0
                toRemove -= thisCount
            else:
                curPyr[thisKey] -= toRemove
                # curPyr[thisKey] = np.floor(curPyr[thisKey] - toRemove)
                toRemove = 0
            
            
            # print(curPyr[keys[k]])
        # print(curPyr)
        # print(overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[1][1]))
        curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[ifrDatabase][1])
        
        allIFRs.append(curIFR)
    ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
    # ax1.plot(100*allVaccRate,allIFRs,linewidth=4,label=ifrs[ifrDatabase][0],color=getColor(counter),ls=lstyles[counter % len(lstyles)])
    # Save results
    allAllIFRs.append(allIFRs)
    
    counter += 1
    
IFRdf = pd.DataFrame(np.array(allAllIFRs))
allMean = IFRdf.mean()    

# ax1.plot(100*allVaccRate,allMean,'k*-',label='Average',linewidth=4,markersize=10)
ax1.plot(100*allVaccRate,allMean,'r*-',label='Average',linewidth=4,markersize=10)

ax1.legend()


ax1.set_ylim(bottom=0)
ax1.set_xlim([0,100])

ax1.set_xlabel('Vaccination rate [%]')
ax1.set_ylabel('Effective IFR [%]')

plt.tight_layout()

# if saveFigures:
#     plt.savefig('figs/VaccinationIFR_DK')
    
# ax1.set_ylim(top=0.25)
# if saveFigures:
#     plt.savefig('figs/VaccinationIFR_DK_Zoom')
    
# ax1.set_ylim(top=0.05)
# if saveFigures:
#     plt.savefig('figs/VaccinationIFR_DK_ZoomZoom')

# ax1.set_yscale('log') 
# ax1.set_ylim(bottom=0.001,top=2)

# ax1.grid(which='minor')
# from matplotlib.ticker import ScalarFormatter
# for axis in [ax1.xaxis, ax1.yaxis]:
#     axis.set_major_formatter(ScalarFormatter())
    
# if saveFigures:
#     plt.savefig('figs/VaccinationIFR_DK_Log')
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Different places

In [13]:
# Age-stratified IFR estimates for COVID-19, as compiled by Marc Bevand
ifrs_covid = [

        # Calculated from Spanish ENE-COVID study
        # (see calc_ifr.py)
        ('ENE-COVID', {
            (0,9):    0.003,
            (10,19):  0.004,
            (20,29):  0.015,
            (30,39):  0.030,
            (40,49):  0.064,
            (50,59):  0.213,
            (60,69):  0.718,
            (70,79):  2.384,
            (80,89):  8.466,
            (90,maxage): 12.497,
        }),

        # US CDC estimate as of 19 Mar 2021
        # https://www.cdc.gov/coronavirus/2019-ncov/hcp/planning-scenarios.html
        # (table 1)
        ('US CDC', {
            (0,17):   0.002,
            (18,49):  0.05,
            (50,64):  0.6,
            (65,maxage): 9.0,
        }),

        # Verity et al.
        # https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30243-7/fulltext
        # (table 1)
        ('Verity', {
            (0,9):    0.00161,
            (10,19):  0.00695,
            (20,29):  0.0309,
            (30,39):  0.0844,
            (40,49):  0.161,
            (50,59):  0.595,
            (60,69):  1.93,
            (70,79):  4.28,
            (80,maxage): 7.80,
        }),

        # Levin et al.
        # https://link.springer.com/article/10.1007/s10654-020-00698-1
        # (table 3)
        ('Levin', {
            (0,34):   0.004,
            (35,44):  0.068,
            (45,54):  0.23,
            (55,64):  0.75,
            (65,74):  2.5,
            (75,84):  8.5,
            (85,maxage): 28.3,
        }),

        # Salje et al.: Estimating the burden of SARS-CoV-2 in France
        # https://science.sciencemag.org/content/369/6500/208
        # Supplementary Materials:
        # https://science.sciencemag.org/content/sci/suppl/2020/05/12/science.abc3517.DC1/abc3517_Salje_SM_rev2.pdf
        # (table S2)
        ('Salje', {
            (0,19):   0.001,
            (20,29):  0.005,
            (30,39):  0.02,
            (40,49):  0.05,
            (50,59):  0.2,
            (60,69):  0.7,
            (70,79):  1.9,
            (80,maxage):  8.3,
        }),

        # # Perez-Saez et al.
        # # https://www.thelancet.com/journals/laninf/article/PIIS1473-3099(20)30584-3/fulltext
        # ('Perez-Saez', {
        #     (5,9):    0.0016,
        #     (10,19):  0.00032,
        #     (20,49):  0.0092,
        #     (50,64):  0.14,
        #     (65,maxage): 5.6,
        # }),

        # # Picon et al.
        # # https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7493765/
        # # (table 2)
        # ('Picon', {
        #     (20,39):  0.08,
        #     (40,59):  0.24,
        #     (60,maxage):  4.63,
        # }),

        # Poletti et al.
        # https://www.eurosurveillance.org/content/10.2807/1560-7917.ES.2020.25.31.2001383
        # (table 1, column "Any time")
        ('Poletti', {
            (0,19):   0,
            (20,49):  0,
            (50,59):  0.46,
            (60,69):  1.42,
            (70,79):  6.87,
            (80,maxage):  18.35,
        }),

        # # Gudbjartsson et al.: Humoral Immune Response to SARS-CoV-2 in Iceland
        # # https://www.nejm.org/doi/full/10.1056/NEJMoa2026116
        # # Supplementary Appendix 1
        # # https://www.nejm.org/doi/suppl/10.1056/NEJMoa2026116/suppl_file/nejmoa2026116_appendix_1.pdf
        # # (table S7)
        # ('Gudbjartsson', {
        #     (0,70):   0.1,
        #     (71,80):  2.4,
        #     (81,maxage): 11.2,
        # }),

        # Public Health Agency of Sweden
        # https://www.folkhalsomyndigheten.se/contentassets/53c0dc391be54f5d959ead9131edb771/infection-fatality-rate-covid-19-stockholm-technical-report.pdf
        # (table B.1)
        ('PHAS', {
            (0,49):   0.01,
            (50,59):  0.27,
            (60,69):  0.45,
            (70,79):  1.92,
            (80,89):  7.20,
            (90,maxage):  16.21,
        }),

        # O’Driscoll et al.: Age-specific mortality and immunity patterns of SARS-CoV-2
        # https://www.nature.com/articles/s41586-020-2918-0
        # Supplementary information
        # https://static-content.springer.com/esm/art%3A10.1038%2Fs41586-020-2918-0/MediaObjects/41586_2020_2918_MOESM1_ESM.pdf
        # (table S3)
        ('O’Driscoll', {
            (0,4):   0.003,
            (5,9):   0.001,
            (10,14): 0.001,
            (15,19): 0.003,
            (20,24): 0.006,
            (25,29): 0.013,
            (30,34): 0.024,
            (35,39): 0.040,
            (40,44): 0.075,
            (45,49): 0.121,
            (50,54): 0.207,
            (55,59): 0.323,
            (60,64): 0.456,
            (65,69): 1.075,
            (70,74): 1.674,
            (75,79): 3.203,
            (80,maxage): 8.292,
        }),

        # # Ward et al.: Antibody prevalence for SARS-CoV-2 in England following first peak of the pandemic: REACT2 study in 100,000 adults
        # # https://www.nature.com/articles/s41467-021-21237-w
        # # (table 2)
        # ('REACT2', {
        #     (15,44):  0.03,
        #     (45,64):  0.52,
        #     (65,74):  3.13,
        #     (75,maxage): 11.64,
        # }),

        # Yang et al.: Estimating the infection fatality risk of COVID-19 in New York City during the spring 2020 pandemic wave
        # https://www.medrxiv.org/content/10.1101/2020.06.27.20141689v2
        # (table 1)
        ('Yang', {
            (0,24):   0.0097,
            (25,44):  0.12,
            (45,64):  0.94,
            (65,74):  4.87,
            (75,maxage): 14.17,
        }),

        # Molenberghs et al.: Belgian Covid-19 Mortality, Excess Deaths, Number of Deaths per Million, and Infection Fatality Rates
        # https://www.medrxiv.org/content/10.1101/2020.06.20.20136234v1
        # (table 6)
        ('Molenberghs', {
            (0,24):   0.0005,
            (25,44):  0.017,
            (45,64):  0.21,
            (65,74):  2.24,
            (75,84):  4.29,
            (85,maxage): 11.77,
        }),

        # Brazeau et al.
        # https://www.imperial.ac.uk/mrc-global-infectious-disease-analysis/covid-19/report-34-ifr/
        # (table 2, column "IFR (%) with Seroreversion")
        ('Brazeau', {
            (0,4):    0.00,
            (5,9):    0.01,
            (10,14):  0.01,
            (15,19):  0.02,
            (20,24):  0.02,
            (25,29):  0.04,
            (30,34):  0.06,
            (35,39):  0.09,
            (40,44):  0.15,
            (45,49):  0.23,
            (50,54):  0.36,
            (55,59):  0.57,
            (60,64):  0.89,
            (65,69):  1.39,
            (70,74):  2.17,
            (75,79):  3.39,
            (80,84):  5.30,
            (85,89):  8.28,
            (90,maxage): 16.19,
        }),

]

In [14]:


curDefPyr = pyramid['Denmark']
fullPop = np.fromiter(curDefPyr.values(), dtype=float).sum()

keys = list(curDefPyr.keys())

# ratioVacc = 0.99

allVaccRate = np.arange(0,0.99,0.01)
# allVaccRate = np.arange(0,0.99,0.2)

allIFRs = []

for ratioVacc in allVaccRate:
    curPyr = curDefPyr.copy()
    toRemoveTot = np.ceil(fullPop * (ratioVacc))
    toRemove = toRemoveTot
    for k in range(len(curPyr)-1,-1,-1):
        thisKey = keys[k]
        thisCount = curPyr[thisKey]
        
        if (toRemove >= thisCount):
            curPyr[thisKey] = 0
            toRemove -= thisCount
        else:
            curPyr[thisKey] -= toRemove
            # curPyr[thisKey] = np.floor(curPyr[thisKey] - toRemove)
            toRemove = 0
        
        
        # print(curPyr[keys[k]])
    # print(curPyr)
    # print(overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[1][1]))
    curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[0][1])
    
    allIFRs.append(curIFR)
    

In [15]:
dkDefPyr.items()


dict_items([((0, 4), 309000.0), ((5, 9), 297000.0), ((10, 14), 337000.0), ((15, 19), 339000.0), ((20, 24), 374000.0), ((25, 29), 402000.0), ((30, 34), 355000.0), ((35, 39), 317000.0), ((40, 44), 361000.0), ((45, 49), 377000.0), ((50, 54), 422000.0), ((55, 59), 388000.0), ((60, 64), 346000.0), ((65, 69), 309000.0), ((70, 74), 351000.0), ((75, 79), 235000.0), ((80, 84), 147000.0), ((85, 89), 78000.0), ((90, 94), 36000.0), ((95, 99), 10000.0), ((100, 100), 1000.0)])

In [16]:
# pyramid
# 'WORLD','Africa','Asia','Europe','Latin America and the Caribbean','Northern America','Oceania'


regionsToShow = ['Denmark','Peru','Brazil','Africa','Asia','Europe','Latin America and the Caribbean','Northern America','Oceania','WORLD']

allVaccRate = np.arange(0,0.99,0.01)
# allVaccRate = np.arange(0,0.99,0.2)

# Save dataframe with data        
# newData = {'VaccRate':allVaccRate,'Denmark':allMean}
newData = {'VaccRate':allVaccRate}
newdf = pd.DataFrame(data=newData)

for curRegion in regionsToShow:
    fig,ax1 = plt.subplots()
    allAllIFRs = []

    # curDefPyr = pyramid['Denmark']
    curDefPyr = pyramid[curRegion]
    fullPop = np.fromiter(curDefPyr.values(), dtype=float).sum()

    lstyles = ('solid', 'dashed', 'dotted', 'dashdot')
    lstyles = ('dashed', 'dotted', 'dashdot')
    markers = ('o', 's', 'v', '^', '<', '>', 'P', '*', 'X', 'D', 'p')


    def getColor(i):
        return plt.cm.twilight((1/20) * i)
        # return plt.cm.bwr((1/10) * i)
        # return plt.cm.bwr((1/20) * i)

    counter = 0

    # All IFRs: ifrs_covid
    # Only some of them: ifrs
    # for ifrDatabase in range(0,1):
    # for ifrDatabase in range(0,len(ifrs)):
    for ifrDatabase in range(0,len(ifrs_covid)):

        allIFRs = []

        for ratioVacc in allVaccRate:
            curPyr = curDefPyr.copy()
            toRemoveTot = np.ceil(fullPop * (ratioVacc))
            toRemove = toRemoveTot
            for k in range(len(curPyr)-1,-1,-1):
                thisKey = keys[k]
                thisCount = curPyr[thisKey]
                
                if (toRemove >= thisCount):
                    curPyr[thisKey] = 0
                    toRemove -= thisCount
                else:
                    curPyr[thisKey] -= toRemove
                    toRemove = 0
                
            # curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[ifrDatabase][1])
            curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs_covid[ifrDatabase][1])
            
            allIFRs.append(curIFR)
            
        # # ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
        ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs_covid[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
        # # ax1.plot(100*allVaccRate,allIFRs,linewidth=4,label=ifrs[ifrDatabase][0],color=getColor(counter),ls=lstyles[counter % len(lstyles)])
        
        # Save results
        allAllIFRs.append(allIFRs)
        
        counter += 1
        
    IFRdf = pd.DataFrame(np.array(allAllIFRs))
    allMean = IFRdf.mean()    

    # ax1.plot(100*allVaccRate,allMean,'k*-',label='Average',linewidth=4,markersize=10)
    ax1.plot(100*allVaccRate,allMean,'r',label='Average',linewidth=4,markersize=10)
    
    # Save results
    newdf[curRegion] = allMean

    ax1.legend()


    ax1.set_ylim(bottom=0)
    ax1.set_xlim([0,100])

    ax1.set_xlabel('Vaccination rate [%]')
    ax1.set_ylabel('Effective IFR [%]')

    ax1.set_title(curRegion)
    ax1.grid()
    plt.tight_layout()

    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion)
            
    ax1.set_ylim(top=0.25)
    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion+'_Zoom')
        
    ax1.set_ylim(top=0.05)
    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion+'_ZoomZoom')

    ax1.set_yscale('log') 
    ax1.set_ylim(bottom=0.001,top=2)

    ax1.grid(which='minor')
    from matplotlib.ticker import ScalarFormatter
    for axis in [ax1.xaxis, ax1.yaxis]:
        axis.set_major_formatter(ScalarFormatter())        
    plt.tight_layout()
        
    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion+'_Log')
        

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
ifrdatafilename = 'IFRofVaccPerRegion.csv'
newdf.to_csv(ifrdatafilename) 

In [18]:
# fig,ax1 = plt.subplots()

# regionsToShow = ['Denmark']

# for curRegion in regionsToShow:
#     allAllIFRs = []

#     # curDefPyr = pyramid['Denmark']
#     curDefPyr = pyramid[curRegion]
#     fullPop = np.fromiter(curDefPyr.values(), dtype=float).sum()

#     lstyles = ('solid', 'dashed', 'dotted', 'dashdot')
#     lstyles = ('dashed', 'dotted', 'dashdot')
#     markers = ('o', 's', 'v', '^', '<', '>', 'P', '*', 'X', 'D', 'p')


#     def getColor(i):
#         return plt.cm.twilight((1/20) * i)
#         # return plt.cm.bwr((1/10) * i)
#         # return plt.cm.bwr((1/20) * i)

#     counter = 0

#     # All IFRs: ifrs_covid
#     # Only some of them: ifrs
#     # for ifrDatabase in range(0,1):
#     # for ifrDatabase in range(0,len(ifrs)):
#     for ifrDatabase in range(0,len(ifrs_covid)):

#         allVaccRate = np.arange(0,0.99,0.01)
#         # allVaccRate = np.arange(0,0.99,0.2)
#         allIFRs = []

#         for ratioVacc in allVaccRate:
#             curPyr = curDefPyr.copy()
#             toRemoveTot = np.ceil(fullPop * (ratioVacc))
#             toRemove = toRemoveTot
#             for k in range(len(curPyr)-1,-1,-1):
#                 thisKey = keys[k]
#                 thisCount = curPyr[thisKey]
                
#                 if (toRemove >= thisCount):
#                     curPyr[thisKey] = 0
#                     toRemove -= thisCount
#                 else:
#                     curPyr[thisKey] -= toRemove
#                     toRemove = 0
                
#             # curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[ifrDatabase][1])
#             curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs_covid[ifrDatabase][1])
            
#             allIFRs.append(curIFR)
            
#         # # ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
#         ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs_covid[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
#         # # ax1.plot(100*allVaccRate,allIFRs,linewidth=4,label=ifrs[ifrDatabase][0],color=getColor(counter),ls=lstyles[counter % len(lstyles)])
        
#         # Save results
#         allAllIFRs.append(allIFRs)
        
#         counter += 1
        
#     IFRdf = pd.DataFrame(np.array(allAllIFRs))
#     allMean = IFRdf.mean()    

#     ax1.plot(100*allVaccRate,allMean,'r',label=curRegion,linewidth=4,markersize=10)

# ax1.legend()


# ax1.set_ylim(bottom=0)
# ax1.set_xlim([0,100])

# ax1.set_xlabel('Vaccination rate [%]')
# ax1.set_ylabel('Effective IFR [%]')

# ax1.set_title('Comparison of world regions')
# ax1.grid()
# plt.tight_layout()

    

In [19]:


regionsToShow = ['Denmark','Peru','Brazil','Africa','Asia','Europe','Latin America and the Caribbean','Northern America','Oceania','WORLD']

for curRegion in regionsToShow:
    fig,ax1 = plt.subplots()
    allAllIFRs = []

    # curDefPyr = pyramid['Denmark']
    curDefPyr = pyramid[curRegion]
    fullPop = np.fromiter(curDefPyr.values(), dtype=float).sum()

    lstyles = ('solid', 'dashed', 'dotted', 'dashdot')
    lstyles = ('dashed', 'dotted', 'dashdot')
    markers = ('o', 's', 'v', '^', '<', '>', 'P', '*', 'X', 'D', 'p')


    def getColor(i):
        return plt.cm.twilight((1/20) * i)
        # return plt.cm.bwr((1/10) * i)
        # return plt.cm.bwr((1/20) * i)

    counter = 0

    # All IFRs: ifrs_covid
    # Only some of them: ifrs
    # for ifrDatabase in range(0,1):
    # for ifrDatabase in range(0,len(ifrs)):
    for ifrDatabase in range(0,len(ifrs_covid)):

        allVaccRate = np.arange(0,0.99,0.01)
        # allVaccRate = np.arange(0,0.99,0.2)
        allIFRs = []

        for ratioVacc in allVaccRate:
            curPyr = curDefPyr.copy()
            toRemoveTot = np.ceil(fullPop * (ratioVacc))
            toRemove = toRemoveTot
            for k in range(len(curPyr)-1,-1,-1):
                thisKey = keys[k]
                thisCount = curPyr[thisKey]
                
                if (toRemove >= thisCount):
                    curPyr[thisKey] = 0
                    toRemove -= thisCount
                else:
                    curPyr[thisKey] -= toRemove
                    toRemove = 0
                
            # curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs[ifrDatabase][1])
            curIFR = overall_ifr(pyramid_region=curPyr,ifr_age_stratified=ifrs_covid[ifrDatabase][1])
            
            allIFRs.append(curIFR)
            
        # # ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
        ax1.plot(100*allVaccRate,allIFRs,linewidth=3,markersize=7,label=ifrs_covid[ifrDatabase][0],color=getColor(counter),marker=markers[counter % len(markers)],ls=lstyles[counter % len(lstyles)])
        # # ax1.plot(100*allVaccRate,allIFRs,linewidth=4,label=ifrs[ifrDatabase][0],color=getColor(counter),ls=lstyles[counter % len(lstyles)])
        
        # Save results
        allAllIFRs.append(allIFRs)
        
        counter += 1
        
    IFRdf = pd.DataFrame(np.array(allAllIFRs))
    allMean = IFRdf.mean()    

    # ax1.plot(100*allVaccRate,allMean,'k*-',label='Average',linewidth=4,markersize=10)
    ax1.plot(100*allVaccRate,allMean,'r',label='Average',linewidth=4,markersize=10)

    ax1.legend()


    ax1.set_ylim(bottom=0)
    ax1.set_xlim([0,100])

    ax1.set_xlabel('Vaccination rate [%]')
    ax1.set_ylabel('Effective IFR [%]')

    ax1.set_title(curRegion)
    ax1.grid()
    plt.tight_layout()

    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion)
            
    ax1.set_ylim(top=0.25)
    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion+'_Zoom')
        
    ax1.set_ylim(top=0.05)
    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion+'_ZoomZoom')

    ax1.set_yscale('log') 
    ax1.set_ylim(bottom=0.001,top=2)

    ax1.grid(which='minor')
    from matplotlib.ticker import ScalarFormatter
    for axis in [ax1.xaxis, ax1.yaxis]:
        axis.set_major_formatter(ScalarFormatter())        
    plt.tight_layout()
        
    if saveFigures:
        plt.savefig('figs/VaccIFRs/VaccinationIFR_'+curRegion+'_Log')
    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-19-3f35035ccfaf>:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig,ax1 = plt.subplots()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …